In [66]:
## Part 1 - getting data
import folium
import requests
import pandas as pd
import numpy as np

crash_table = pd.read_csv("https://cmsc320.github.io/files/Baltimore_City_Vehicle_Crashes.csv")
crash_table


/var/folders/cz/ztsjvl7d28s0fjqn9dd9sxmh0000gn/T/ipykernel_5770/1608876202.py:7: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  crash_table = pd.read_csv("https://cmsc320.github.io/files/Baltimore_City_Vehicle_Crashes.csv")


,YEAR,QUARTER,LIGHT_DESC,LIGHT_CODE,COUNTY_NO,MUNI_DESC,MUNI_CODE,JUNCTION_DESC,JUNCTION_CODE,COLLISION_TYPE_DESC,...,FEET_MILES_FLAG_DESC,FEET_MILES_FLAG,DISTANCE_DIR_FLAG,REFERENCE_NO,REFERENCE_TYPE_CODE,REFERENCE_SUFFIX,REFERENCE_ROAD_NAME,LATITUDE,LONGITUDE,LOCATION
0,2020,Q2,NaN,6.02,24.0,NaN,NaN,Non Intersection,1.0,Other,...,Miles,M,N,NaN,NaN,NaN,NORTH AVE,39.311025,-76.616429,POINT (-76.616429453205 39.311024794431)
1,2017,Q2,Daylight,1.00,24.0,NaN,NaN,NaN,NaN,Single Vehicle,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.282928,-76.635215,POINT (-76.6352150952347 39.2829284750108)
2,2020,Q2,Daylight,1.00,24.0,NaN,NaN,Intersection,2.0,Other,...,Feet,F,S,NaN,NaN,NaN,WINDSOR AVE,39.312903,-76.651472,POINT (-76.651471912939 39.312903404529)
3,2020,Q2,Daylight,1.00,24.0,NaN,NaN,NaN,99.0,Same Movement Angle,...,NaN,U,E,NaN,NaN,NaN,WASHINGTON ST,39.294944,-76.599329,POINT (-76.599328693204 39.294943770185)
4,2020,Q2,NaN,5.02,24.0,NaN,NaN,NaN,NaN,Same Direction Rear End,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.296874,-76.595871,POINT (-76.595871121891 39.296873988072)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135585,2021,Q4,Unknown,99.00,24.0,NaN,NaN,Non Intersection,1.0,Same Direction Sideswipe,...,Feet,F,E,NaN,NaN,NaN,LINDEN AVE.,39.315169,-76.635210,POINT (-76.635209753929 39.315168933901)
135586,2021,Q4,Daylight,1.00,24.0,NaN,NaN,Intersection,2.0,Single Vehicle,...,Feet,F,N,NaN,NaN,NaN,N CALVERT ST,39.291235,-76.612507,POINT (-76.61250671377 39.291234751225)
135587,2021,Q4,Dark Lights On,3.00,24.0,NaN,NaN,Intersection,2.0,Same Direction Rear End,...,Feet,F,S,NaN,NaN,NaN,E FAYETT,39.290660,-76.606670,POINT (-76.60667 39.29066)
135588,2021,Q4,Dark Lights On,3.00,24.0,NaN,999.0,Non Intersection,1.0,Same Direction Rear End Left Turn,...,Feet,F,W,2125.0,MU,NaN,GARRISON BLVD,39.343406,-76.640647,POINT (-76.640646523492 39.343405723453)


In [67]:
crash_table[pd.notnull(crash_table["LOCATION"])]["LOCATION"].count()

# Since the data set contains so many incidents, and each incidents all have location values throughout
# 135590 rows, I decided to only use the first 500 entries in the set so that the map does not get 
# completely cluttered by markers


135590

In [68]:
## Part 2 - Making a Map
map_osm = folium.Map(location=[39.29, -76.61], zoom_start=13)
map_osm

In [69]:
## Part 3 - combining part1 and part2

# Using columns to find which list I can use to decide what I want to mark for in map
crash_table.columns

data = crash_table.head(500)
accidents = sorted(list(set(data.COLLISION_TYPE_DESC)))
accidents

# Going to use COLLISION_TYPE_DESC, ACC_DATE, ACC_TIME, and WEATHER_DESC.
# I choose to work on the map depends on the Collision type because by doing so, when we see the points on map,
# we can see how a specific location is easy to get a certain collision compare to others. Later in the code,
# I will catrgorize the collision description into each catrgory for use in map.


['Angle Meets Left Turn',
 'Angle Meets Right Turn',
 'Head On',
 'Head On Left Turn',
 'Not Applicable',
 'Opposite Direction Both Left Turn',
 'Opposite Direction Sideswipe',
 'Other',
 'Same Direction Both Left Turn',
 'Same Direction Left Turn',
 'Same Direction Rear End',
 'Same Direction Rear End Left Turn',
 'Same Direction Rear End Right Turn',
 'Same Direction Right Turn',
 'Same Direction Sideswipe',
 'Same Movement Angle',
 'Single Vehicle',
 'Unknown']

In [70]:
# Part 3 - combining part 1 and part2

# Assigning each entry in the data selected above (Collision description) for coloring on the map
categories = ['Angle Meets','Head On','Other','Opposite Direction','Same Direction','Single Vehicle','Same Movement']

# creating a dictionary assigning each accident to their categories
accident_key = {'Angle Meets Left Turn' : 'Angle Meets',
 'Angle Meets Right Turn'  : 'Angle Meets',
 'Head On' : 'Head On',
 'Head On Left Turn' : 'Head on',
 'Not Applicable' : 'Other',
 'Opposite Direction Both Left Turn' : 'Opposite Direction',
 'Opposite Direction Sideswipe' : 'Opposite Direction',
 'Other' : 'Other',
 'Same Direction Both Left Turn' : 'Same Direction',
 'Same Direction Left Turn' : 'Same Direction',
 'Same Direction Rear End'  : 'Same Direction',
 'Same Direction Rear End Left Turn'  : 'Same Direction',
 'Same Direction Rear End Right Turn'  : 'Same Direction',
 'Same Direction Right Turn'  : 'Same Direction',
 'Same Direction Sideswipe'  : 'Same Direction',
 'Same Movement Angle'  : 'Same Movement',
 'Single Vehicle' : 'Single Vehicle',
 'Unknown' : 'Other'}

column = []
for index, row in data.iterrows():
    column.append(accident_key[row.COLLISION_TYPE_DESC])
data.insert(len(data.columns),'Collision_Category',column)

map_osm.add_child(folium.LatLngPopup())

colors = ['red', 'blue', 'green', 'purple', 'orange', 'beige', 'darkpurple']
weather_key = {np.nan : 'Unkown', 'Clear' : 'Sunny', 'Unknown' : 'Unknown' , 'Raining' : 'Raining', 
'Severe Winds' : 'Windy', 'Other' : 'Other', 'Not Applicable' : 'Unkown'}


for category,color in zip(categories,colors):
    new_data = data[data['Collision_Category'] == category]
    for index,row in new_data.iterrows():
        popup = folium.Popup(html = 'Accident Location: %s<br> Accident Date: %s<br> Accident Time: %s<br> Weather: %s<br>Collision Type: %s<br> Collision Description: %s' % (row.REFERENCE_ROAD_NAME, row.ACC_DATE, row.ACC_TIME, weather_key[row.WEATHER_DESC], row.Collision_Category, row.COLLISION_TYPE_DESC), max_width=400)
        folium.Marker(location = [row.LATITUDE,row.LONGITUDE], popup = popup, icon = folium.Icon(color = color)).add_to(map_osm)
    
map_osm

In [ ]:
# I decided to show the different accident types to see if there are any patterns in certain location which happens to have a specific type of
# accident over another. I also included the weather and time for every accident appearing in map because I think those can take in consideration
# of whether bad wheather, late night, or early morning will cause more accident than other times.

# Since there are many different descriptions in the COLLISION_TYPE_DESC column I decided to group some of them together.
# The categories I grouped them into are: Angle Meets','Head On','Other','Opposite Direction','Same Direction','Single Vehicle','Same Movement'.
# I paired each unique description found in the column with the category I created, and used that to generate the different colored markers on the map.

# One thing that I found interesting is there are alot of orange and dark purple icon at inner city which both represents the accident happend in the 
# same direction. The location of the accident is also very close to traffic lights, which means same direction collision and traffic light position
# has positive correlation which means it will be more likely to have an accident like that in city area. 

# Another thing I found is how almost all of the pink/purple icon happens at the outter city. They represent opposite direction and head on collisoin, 
# which although there are only a few accidents, but they all gather outside of the city. At the same time there are a lot of 'Others' surronding the map
# which is usually car breakdown, and it represents random and is proved by the map that outage could happen at every locaiton.